In [33]:
from bs4 import BeautifulSoup
from datetime import datetime, date
import numpy as np
import pandas as pd
import urllib
import matplotlib.pyplot as plt
import re

In [2]:
# with open('feed.xml', encoding='utf-8') as fp:
#     soup = BeautifulSoup(fp, 'lxml-xml')

## Download content

In [3]:
with urllib.request.urlopen('http://thoibao.com/category/doi-song/hop-thu-tim-ban-doi-song/feed/') as f:
    xmlContent = f.read()
    soup = BeautifulSoup(xmlContent, 'lxml-xml')

In [91]:
arr = []
agePattern1 = re.compile(r'(\d+) tuổi', re.I)
agePattern2 = re.compile(r'ngoài (\d+)', re.I)
agePattern3 = re.compile(r'sinh năm (\d+)', re.I)

for item in soup.find_all('item'):
    dateString = item.find('pubDate').text
    content = item.find('content:encoded').text
    
    contentSoup = BeautifulSoup(content, 'lxml')
    for person in contentSoup.find_all('p'):
        id = person.find('strong')
        try:
            id = int(id.text)
            description = person.text.split('–')[-1].strip()
            
            age = None
            match = agePattern1.search(description)
            age = None if match is None else int(match.group(1))
            
            if age is None:
                match = agePattern2.search(description)
                age = None if match is None else int(match.group(1))
            if age is None:
                match = agePattern3.search(description)
                age = date.today().year - int(match.group(1))
            
            arr.append((dateString, id, age, description))
        except (ValueError, AttributeError):
            pass

In [92]:
df = pd.DataFrame(arr, columns=['date', 'id', 'age', 'description'])
df['date'] = df['date'].astype(np.datetime64).apply(lambda d: d.date())
df.shape

(332, 4)

## Compile statistics

In [93]:
grp = df.groupby('id')
df2 = pd.DataFrame({
    'age': grp['age'].mean(),
    'min': grp['date'].min(),
    'max': grp['date'].max(),
    'count': grp['date'].count(),
    'description': grp['description'].min()
})

In [94]:
def describe(id):
    try:
        _id = id if type(id) is list else [id]
        people = df2[df2.index.isin(_id)]
    except KeyError:
        print(f'Cannot find id {id}')
        return

    for id, person in people.iterrows():
        print(f'''
id = {id}. From {person['min']:%b %d, %Y} to {person['max']:%b %d, %Y}. {person['count']} kỳ.
{person['description']}
''')

## Query

New listings:

In [95]:
allDates = np.sort(df['date'].unique())
df2[df2['min'] >= allDates[-4]]

,age,min,max,count,description
id,,,,,
2412,50,2018-10-19,2018-11-09,4,"Nam, tuổi ngoài 50, cao 1M 78, nặng 70 Kg, tốt..."
2413,46,2018-10-19,2018-11-09,4,"Nam, 46 tuổi, cao 1M 71, độc thân, không cờ bạ..."
2414,58,2018-11-03,2018-11-09,2,"Nữ, 58 tuổi, người miền Nam, sống và làm việc ..."
2415,33,2018-11-09,2018-11-09,1,"Mình là nữ, 33 tuổi, độc thân, người miền Nam...."
2416,41,2018-11-09,2018-11-09,1,"Nữ sinh năm 1977, chưa từng kết hôn, chưa có c..."


Contacts:

In [76]:
df2[df2.index.isin([2357, 2379, 2384])]

,age,min,max,count,description
id,,,,,
2379,29,2018-09-07,2018-11-09,10,"Nữ, 29 tuổi, ở Toronto, ngoại hình xinh xắn. c..."
2384,28,2018-09-07,2018-11-09,10,"Nữ, 28 tuổi chưa từng lập gia đình, ngoan hiền..."


In [77]:
describe([2379, 2384])


id = 2379. From Sep 07, 2018 to Nov 09, 2018. 10 kỳ.
Nữ, 29 tuổi, ở Toronto, ngoại hình xinh xắn. cao 1M 57, thật thà, vui vẻ. Tốt nghiệp đại học. Công việc ổn định, thích travel, music, movie. Mong tìm bạn trai tuổi từ 30-36, ngoại hình cao ráo, dễ nhìn, chân thật. Không hút thuốc, tốt nghiệp đại học, công việc ổn. Nếu thích hợp sẽ tiến xa hơn, xin cho số phone hoặc email để liên lạc.


id = 2384. From Sep 07, 2018 to Nov 09, 2018. 10 kỳ.
Nữ, 28 tuổi chưa từng lập gia đình, ngoan hiền, ít nói, hiểu biết, chu đáo trong mọi công việc, đang có việc làm ổn định và rất yêu thích mái ấm gia đình. Mong tìm bạn trai từ 30 tuổi trở lên, nếu có nhã ý xin cho số phone, email để tiện liên lạc. Rất mong nhận được thư của người có thành ý. Cám ơn.



## Playground

In [79]:
df2['age'].describe()

count    41.000000
mean     45.073171
std      12.863495
min      21.000000
25%      34.000000
50%      48.000000
75%      55.000000
max      65.000000
Name: age, dtype: float64

In [81]:
df2.shape

(41, 5)

In [85]:
df2[df2.age <= 30]

,age,min,max,count,description
id,,,,,
2370,21,2018-09-07,2018-09-07,1,"Nữ, sinh năm 1997, Châu Thành tỉnh Bến Tre,Việ..."
2375,21,2018-09-07,2018-10-26,8,"Tôi có một cháu gái, 21 tuổi quê ở Gò Công. Tr..."
2379,29,2018-09-07,2018-11-09,10,"Nữ, 29 tuổi, ở Toronto, ngoại hình xinh xắn. c..."
2380,25,2018-09-07,2018-11-09,10,"Nữ, 25 tuổi, độc thân, người miền Nam, cao 1M ..."
2384,28,2018-09-07,2018-11-09,10,"Nữ, 28 tuổi chưa từng lập gia đình, ngoan hiền..."
2398,27,2018-09-07,2018-11-09,10,"Tôi có một cháu gái, 27 tuổi, hiện đang ở Sài ..."
2400,29,2018-09-07,2018-11-09,10,"Tôi có một cháu gái 29 tuổi, có ngoại hình cao..."
2406,25,2018-09-07,2018-11-09,10,"Tôi có một cháu trai 25 tuổi, người Canada, Tà..."
